In [1]:
import pickle
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlrun

# Train the model
iris = datasets.load_iris()
X, y = iris.data, iris.target
model = RandomForestClassifier()
model.fit(X, y)

# Save the model to V3IO
file_path = "v3io:///users/normal-user/model.pkl"
with mlrun.get_dataitem(file_path).open("wb") as file:
    pickle.dump(model, file)

In [2]:
import mlrun

project_name = "artifacts-proj"
project = mlrun.get_or_create_project(name=project_name)

> 2025-03-18 14:45:02,279 [info] Project loaded successfully: {"project_name":"artifacts-proj"}


In [3]:
%%writefile ./function.py
import mlrun
import os
import pandas as pd
from itertools import cycle
from random import randint, choice
from sklearn.datasets import make_classification, load_iris
from uuid import uuid4

BYTE = bytearray(1)
ONE_KB = bytearray(BYTE * 1024)
ONE_MB = bytearray(ONE_KB * 1024)


def gen_id():
    return str(uuid4())[:8]

def create_all_artifacts(context, artifact_size, model_size, rows, columns, run_id, tag=None, *args, **kwargs):
    # DATASET
    total_features = columns
    n_informative = min(10, total_features)
    n_redundant = min(columns - 1, total_features - n_informative)
    n_repeated = 0
    X, y = make_classification(
        n_samples=rows,
        n_features=total_features,
        n_informative=n_informative,
        n_redundant=n_redundant,
        random_state=42,
    )
    df = pd.DataFrame(X)
    df.columns = [f"feature_{i}" for i in range(1, total_features + 1)]
    _dataset = f"{run_id}-dataset"
    context.log_dataset(_dataset, df=df, format="csv", tag=tag)

    # ARTIFACT
    artifact_file = f"{run_id}_data.txt"
    data = ONE_KB * artifact_size
    _artifact = f"{run_id}-artifact"
    context.log_artifact(_artifact, body=data, local_path=artifact_file, tag=tag)

    # MODEL
    iris = load_iris()
    _model = f"{run_id}-model"
    context.log_model(
        _model,
        model_file="v3io:///users/normal-user/model.pkl",
        training_set=pd.DataFrame(iris.data, columns=iris.feature_names),
        framework="sklearn",
        upload=False,
        tag=tag
    )



def hyper_func(
    context, run_id, p1=None, p2=None, artifact_size=None, model_size=None, rows=None, columns=None, *args, **kwargs
):
    # ALL ARTIFACTS
    create_all_artifacts(context, artifact_size, model_size, rows, columns, run_id, *args, **kwargs)
    
    # HYPERPARAMS
    if p1 is None or p2 is None:
        print("Skipping hyperparameter calculation as p1 or p2 is None.")
        return
    
    print(f"p1={p1}, p2={p2}, result={p1 * p2}")
    context.log_result("multiplier", p1 * p2)

Overwriting ./function.py


In [4]:
import string
from uuid import uuid4
from random import choice


def gen_id():
    return str(uuid4())[:8]


def set_create_all_arts():
    prefix = "".join(choice(string.ascii_letters) for _ in range(5))
    name = f"{prefix}-{gen_id()}"
    func = project.set_function(
        name=name,
        func="./function.py",
        image="mlrun/mlrun",
        kind="job",
        handler="create_all_artifacts",
    )
    return func

In [5]:
def run_twice(params_dict, watch=False, schedule=None):
    func = set_create_all_arts()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-twice"
    for i in range(2):
        func.run(params=params, watch=watch, schedule=schedule)

In [6]:
# run_twice(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )

In [7]:
def run_twice_2(params_dict, watch=False, schedule=None):
    func = set_create_all_arts()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-twice-2"
    params["tag"] = "unique_1"
    func.run(params=params, watch=watch, schedule=schedule)
    params["tag"] = "unique_2"
    func.run(params=params, watch=watch, schedule=schedule)

In [8]:
# run_twice_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )

In [9]:
def run_twice_3(params_dict, watch=False, schedule=None):
    func = set_create_all_arts()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-twice-3"
    func.run(params=params, watch=watch, schedule=schedule)
    params["tag"] = "unique_1"
    func.run(params=params, watch=watch, schedule=schedule)
    params["tag"] = None
    func.run(params=params, watch=watch, schedule=schedule)

In [10]:
# run_twice_3(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [11]:
def run_twice_4(params_dict, watch=False, schedule=None):
    func = set_create_all_arts()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-twice-4"
    params["tag"] = "unique_1"
    func.run(params=params, watch=watch, schedule=schedule)
    params["tag"] = None
    func.run(params=params, watch=watch, schedule=schedule)

In [12]:
# run_twice_4(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [13]:
import mlrun
import string
from uuid import uuid4
from random import choice


def gen_id():
    return str(uuid4())[:8]


def set_hyper_func():
    prefix = "".join(choice(string.ascii_letters) for _ in range(5))
    name = f"{prefix}-{gen_id()}"
    grid_params = {"p1": [2, 4, 1], "p2": [10, 20, 30]}
    task = mlrun.new_task(f"{name}-hyper-func").with_hyper_params(
        grid_params, selector="max.multiplier"
    )
    func = mlrun.code_to_function(
        name=name,
        filename="./function.py",
        kind="job",
        handler="hyper_func",
    )
    project.set_function(func)
    return func, name, task

In [14]:
def run_hyper_job(params_dict, watch=False, schedule=None):
    func, name, task = set_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-hyper"
    func.run(task, params=params, watch=watch, schedule=schedule)

In [15]:
# run_hyper_job(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [16]:
def run_hyper_job_2(params_dict, watch=False, schedule=None):
    func, name, task = set_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-hyper-2"
    params["tag"] = "unique_1"
    func.run(task, params=params, watch=watch, schedule=schedule)

In [17]:
# run_hyper_job_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [18]:
def run_hyper_job_3(params_dict, watch=False, schedule=None):
    func, name, task = set_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-hyper-3"
    params["tag"] = "unique_1"
    func.run(task, params=params, watch=watch, schedule=schedule)
    params["tag"] = None
    func.run(task, params=params, watch=watch, schedule=schedule)
    func.run(task, params=params, watch=watch, schedule=schedule)

In [19]:
# run_hyper_job_3(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [20]:
def run_simple_and_hyper_2(params_dict, watch=False, schedule=None):
    func, name, task = set_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-simple-hyper-2"
    func.run(
        name=f"{name}-simple-hyper-2", params=params, watch=watch, schedule=schedule
    )
    func.run(
        task,
        name=f"{name}-simple-hyper-2",
        params=params,
        watch=watch,
        schedule=schedule,
    )

In [21]:
# run_simple_and_hyper_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [22]:
def run_simple_and_hyper(params_dict, watch=False, schedule=None):
    func, name, task = set_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-simple-hyper"
    func.run(
        task, name=f"{name}-simple-hyper", params=params, watch=watch, schedule=schedule
    )
    func.run(name=f"{name}-simple-hyper", params=params, watch=watch, schedule=schedule)

In [23]:
# run_simple_and_hyper(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [24]:
import string
from uuid import uuid4
from random import choice


def gen_id():
    return str(uuid4())[:8]


def set_br_hyper_func():
    prefix = "".join(choice(string.ascii_letters) for _ in range(5))
    name = f"{prefix}-{gen_id()}"
    func = project.set_function(
        name=f"{prefix}-{gen_id()}",
        tag="latest",
        func="./function.py",
        image="mlrun/mlrun",
        kind="job",
        handler="hyper_func",
    )
    return func, name

In [25]:
def run_br_hyper_job(params_dict, watch=False, schedule=None):
    func, name = set_br_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-br-hyper"
    func.run(
        name=f"{name}-br-hyper",
        params=params,
        hyperparams={"p1": [2, 4, 1], "p2": [10, 20, 30]},
        selector="max.multiplier",
        watch=watch,
        schedule=schedule,
    )

In [26]:
# run_br_hyper_job_1(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [27]:
def run_br_hyper_job_2(params_dict, watch=False, schedule=None):
    func, name = set_br_hyper_func()
    params = params_dict.get("params", {}).copy()
    params["run_id"] = "run-br-hyper-2"
    func.run(
        name=f"{name}-br-simple-hyper", params=params, watch=watch, schedule=schedule
    )
    params["tag"] = "unique_1"
    func.run(
        name=f"{name}-br-simple-hyper",
        params=params,
        hyperparams={"p1": [2, 4, 1], "p2": [10, 20, 30]},
        selector="max.multiplier",
        watch=watch,
        schedule=schedule,
    )

In [28]:
# run_br_hyper_job_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=True,
#     schedule=False,
# )

In [29]:
# run_twice(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_twice_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_twice_3(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_twice_4(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_hyper_job(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_hyper_job_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_hyper_job_3(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_simple_and_hyper_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_simple_and_hyper(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_br_hyper_job(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )
# run_br_hyper_job_2(
#     {"params": {"artifact_size": 5, "model_size": 0, "rows": 100, "columns": 10}},
#     watch=False,
#     schedule=False,
# )

In [30]:
import time
import random
from concurrent.futures import ThreadPoolExecutor

# Helper function to run jobs immediately


def job_function(job, params_dict, watch):
    job(params_dict, watch=watch)


# Function to run jobs immediately


def run_jobs():
    # Define the jobs and their parameters
    total_runs = 5

    # Define the job configurations (no scheduling)
    job_configs = [
        (
            run_twice,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_twice_2,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_twice_3,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_twice_4,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_hyper_job,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_hyper_job_2,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_hyper_job_3,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_simple_and_hyper_2,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_simple_and_hyper,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_br_hyper_job,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
        (
            run_br_hyper_job_2,
            {
                "params": {
                    "artifact_size": 5,
                    "model_size": 0,
                    "rows": 100,
                    "columns": 10,
                }
            },
            False,
        ),
    ]

    # Use ThreadPoolExecutor to run the jobs concurrently
    with ThreadPoolExecutor(max_workers=4) as executor:
        for _ in range(total_runs):
            for job, params_dict, watch in job_configs:
                # Submit the job for immediate execution
                executor.submit(job_function, job, params_dict, watch)

            random_delay = random.randint(10, 18)
            time.sleep(random_delay)

In [31]:
# Run the jobs
run_jobs()

> 2025-03-18 14:45:04,360 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"ekeaf-28fa0077-create-all-artifacts","uid":"0cdc6809a1114477a1a29955e9574c1e"}
> 2025-03-18 14:45:04,798 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"svkyc-8c0791bf-create-all-artifacts","uid":"9685ade491db442a999a53563f60e408"}
> 2025-03-18 14:45:04,833 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"brqpl-3e40ffb4-create-all-artifacts","uid":"1ba9201df99c4676b2b4bd04d0bcdb5c"}
> 2025-03-18 14:45:05,632 [info] Job is running in the background, pod: ekeaf-28fa0077-create-all-artifacts-vrqsm


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
artifacts-proj,...e9574c1e,0,Mar 18 12:45:05,NaT,running,,ekeaf-28fa0077-create-all-artifacts,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,artifact_size=5model_size=0rows=100columns=10run_id=run-twice-3,


> 2025-03-18 14:45:05,674 [info] Run execution finished: {"name":"ekeaf-28fa0077-create-all-artifacts","status":"running"}
> 2025-03-18 14:45:06,030 [info] Job is running in the background, pod: brqpl-3e40ffb4-create-all-artifacts-6rwwn


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
artifacts-proj,...d0bcdb5c,0,Mar 18 12:45:06,NaT,running,,brqpl-3e40ffb4-create-all-artifacts,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,artifact_size=5model_size=0rows=100columns=10run_id=run-twice-2tag=unique_1,


> 2025-03-18 14:45:06,043 [info] Run execution finished: {"name":"brqpl-3e40ffb4-create-all-artifacts","status":"running"}
> 2025-03-18 14:45:06,046 [info] Job is running in the background, pod: svkyc-8c0791bf-create-all-artifacts-pp8q7


...

> 2025-03-18 14:45:39,041 [info] Run execution finished: {"name":"nkqph-66bf369b-hyper-func","status":"running"}
> 2025-03-18 14:45:39,711 [info] Job is running in the background, pod: oawcm-db4ed9f4-simple-hyper-s8t6n


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
artifacts-proj,...b27cd8d6,0,Mar 18 12:45:39,NaT,running,,oawcm-db4ed9f4-simple-hyper,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,artifact_size=5model_size=0rows=100columns=10run_id=run-simple-hyper,


> 2025-03-18 14:45:39,726 [info] Run execution finished: {"name":"oawcm-db4ed9f4-simple-hyper","status":"running"}
> 2025-03-18 14:45:46,328 [error] error getting build status: details: AttributeError("'NoneType' object has no attribute 'encode'"), caused by: 500 Server Error: Internal Server Error for url: https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com/api/v1/build/status?name=nkqph-66bf369b&project=artifacts-proj&tag=latest&logs=no&offset=0&events_offset=0&last_log_timestamp=0.0&verbose=no
> 2025-03-18 14:45:46,330 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"nkqph-66bf369b-hyper-func","uid":"ab30164669b94199acfa270c3b7b2523"}
> 2025-03-18 14:45:47,527 [info] Job is running in the background, pod: nkqph-66bf369b-hyper-func-flt9q


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
artifacts-proj,...3b7b2523,0,Mar 18 12:45:47,NaT,running,,nkqph-66bf369b-hyper-func,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,artifact_size=5model_size=0rows=100columns=10run_id=run-hyper-3tag=None,


> 2025-03-18 14:45:47,549 [info] Run execution finished: {"name":"nkqph-66bf369b-hyper-func","status":"running"}
